![TensorFlow](notebook_diagrams/tensorflow.png)

# Tutorial for TensorFlow
TensorFlow is a library for creating scalable deep learning models that are efficient and compact.  It is widely used across deep learning applications, and is actually used as the backend for Keras.  Compared to **PyTorch** and **Keras**, it is the most computationally-efficient library of these three popular frameworks, and because of this it is quite often used for deploying models.

**NOTE**: TensorFlow 2.0 just came

The key element behind TensorFlow's efficiency is the [computation graph](https://medium.com/ai%C2%B3-theory-practice-business/tensorflow-1-0-vs-2-0-part-1-computational-graphs-4bb6e31c1a0f).  This is a directed graph where different nodes (circles on the diagram below) represent different TensorFlow operations, and edges (arrows on the diagram below) represent tensors "flowing" between operations.

![tf-comp-graph](notebook_diagrams/tf_comp_graph.png)

## 1. Import Block
**NOTE**: We'll be using TensorFlow 1.15, CPU-only, for this tutorial, but TensorFlow 2.0 is the most recent version of this package, and therefore contains the most advanced capabilities.

TensorFlow is a large Python package, particularly when cuda (which is used when making GPU computations) bindings are installed.  If you know you will only be using a CPU for TensorFlow, it is advisable that you install a CPU-only version of this package.

**Known Windows 8 Issue**: If you are running this on Windows 8, please see [this Stack Overflow post](https://stackoverflow.com/questions/46736219/installing-tensorflow-gpu-on-win-8).

In [ ]:
# Install TensorFlow
!pip install tensorflow_cpu==1.15

# Import TensorFlow
import tensorflow as tf  # Don't need to do "as tf", but typically just done out of convention
tf.enable_eager_execution()


# Install/import other image processing libraries
! pip install Pillow
! pip install --upgrade scipy
! pip install --upgrade scikit-learn
! pip install imageio

import matplotlib.pyplot as plt
import numpy as np
import scipy
import cv2 as cv

# For file paths
import os

import warnings
warnings.filterwarnings('ignore')

## 2. Introduction to the Tensor Data Type
In addition to the computation graph, TensorFlow is also able to create efficient code for deep learning using the `tensor` data type, an object they created that optimizes the flow of information through computation graphs.  This object is similar to the numpy `nd_array` object we saw with numpy, and is used to store and transform numerical data through different computation graphs/deep learning pipelines.

A TensorFlow `tensor` has two different properties:

1. A data type (e.g. `int32`, `float32`)

2. A shape (which also determines **Rank**, or the number of dimensions of a `tensor` object).

Like numpy `nd_array` objects, TensorFlow `tensor` objects must have the same data type for all elements in the tensor.  There are many kinds of `tf.tensor` objects, but the only **mutable** (changeable) of these objects is `tf.Variable`.

#### Rank 0 Tensors

In [ ]:
# Let's create some TensorFlow tensors!
mammal = tf.Variable("Elephant", tf.string)
ignition = tf.Variable(451, tf.int16)
floating = tf.Variable(3.14159265359, tf.float64)
its_complicated = tf.Variable(12.3 - 4.85j, tf.complex64)

#### Rank 1 Tensors

In [ ]:
# Let's create some TensorFlow tensors!
mystr = tf.Variable(["Hello"], tf.string)
cool_numbers  = tf.Variable([3.14159, 2.71828], tf.float32)
first_primes = tf.Variable([2, 3, 5, 7, 11], tf.int32)
its_very_complicated = tf.Variable([12.3 - 4.85j, 7.5 - 6.23j], tf.complex64)

#### Rank 2 Tensors

In [ ]:
# Let's create some TensorFlow tensors!
mymat = tf.Variable([[7],[11]], tf.int16)
myxor = tf.Variable([[False, True],[True, False]], tf.bool)
linear_squares = tf.Variable([[4], [9], [16], [25]], tf.int32)
squarish_squares = tf.Variable([ [4, 9], [16, 25] ], tf.int32)
rank_of_squares = tf.rank(squarish_squares)
mymatC = tf.Variable([[7],[11]], tf.int32)

For reference, here is a table showing how different inputs lead to different `tensor` ranks:
    
![Tensor Rank Table](notebook_diagrams/tensorflow_rank_table.png)

## 3. Basic Math Operations with TensorFlow Tensors
Like numpy `nd_array` objects, we can also use `tf.tensor` objects for doing mathematical operations in Python.

In [ ]:
# Example 1
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

# Operator overloading is also supported
print(tf.square(2) + tf.square(3))

In [ ]:
# Example 2
x = tf.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

## 4. Converting Between Numpy and TensorFlow
Numpy and TensorFlow are quite compatible with each other, which is extremely useful particularly when we can pre-process our data with numpy and/or OpenCV and then load this pre-processed data into TensorFlow.

Calling the method `.numpy()` on a `tf.tensor` object (e.g. `tf.tensor.numpy()`) will convert the data type to a numpy `nd_array` (this could be relevant if we wanted to do post-processing of our data in numpy or openCV.

In [ ]:
import numpy as np

ndarray = np.ones([3, 3])


print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.multiply(ndarray, 42)
print(tensor)


print("And NumPy operations convert Tensors to numpy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

## 5. Neural Network Models in TensorFlow
Neural network models in TensorFlow are similar to what we've seen in Keras (since Keras is built on top of TensorFlow).  We'll explore two aspects of creating your own models with TensorFlow below:

1. Creating custom layers

2. Creating models using composition of layers

### 5.1 Creating Custom Layers

Material referenced from this [TensorFlow tutorial](https://www.tensorflow.org/tutorials/customization/custom_layers).

The best way to implement your own layer is extending the `tf.keras.Layer` class and implementing the constructor method `* __init__` .

In [ ]:
class MyDenseLayer(tf.keras.layers.Layer):  # Notice how we use Keras here!
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_variable("kernel",
                                    shape=[int(input_shape[-1]),
                                           self.num_outputs])

  def call(self, input):
    return tf.matmul(input, self.kernel)

# Make an instance of this layer
layer = MyDenseLayer(10)

# Now, make sure we call layer on something to ".build" it (we can ignore output).
_ = layer(tf.zeros([10, 5])) # Calling the layer `.builds` it.

### 5.2 Creating Custom Models

Material also referenced from [this TensorFlow tutorial](https://www.tensorflow.org/tutorials/customization/custom_layers).

We can use the framework outlined in the class definition below to implement our own custom models.  Below, we will look at the mechanics of creating a **ResNet** block (as visualized below).

![ResNet identity block](notebook_diagrams/resnet_identity_block.png)

**NOTE**: The block of code below is meant only to serve as an example of how we can create different models, and isn't one you have to know specifically.  If you're interested in creating your own custom model, you can use the **general** structure of the code block: the `__init__` and `call` methods.

In [ ]:
class ResnetIdentityBlock(tf.keras.Model):
  
  # This is called the constructor method, and determines what happens when we "instantiate this object"
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  # This function defines how an input is mapped into a prediction
  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)

# Create an instance of this model
block = ResnetIdentityBlock(1, [1, 2, 3])

#### Viewing a Model Summary
Since the models we're calling are Keras models, we can again simply call `model.summary()` to view important parameters and characteristics about the models we create.

In [ ]:
# Make sure we build the model first - can call it on an arbitrary input of correct input size
_ = block(tf.zeros([1, 2, 3, 3])) 

# Now we can summarize
block.summary()

### 5.3 Creating Models in TensorFlow Using Keras
If we aren't creating custom models or layers in TensorFlow, it usually makes more sense to just create our models using Keras.  This is identical to what we saw before (with the exception of placing a `tf.` in front of every keras object).

In [ ]:
# Define the model in the exact same way as we did before!
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])

# Remember to build the model
my_seq(tf.zeros([1, 2, 3, 3]))

# Now we can summarize/train the model
my_seq.summary()

## 6. Training in TensorFlow
Training, in particular custom training, is another reason to consider using TensorFlow for your next machine learning project.  In this section, we will explore an example of training both a pre-trained **Convolutional Neural Network (CNN)**.  We will walk through each section.

### 6.1 Import Statements for ConvNet Example

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Make easier to read
keras = tf.keras

# Use for loading datasets
!pip install tensorflow_datasets
import tensorflow_datasets as tfds
tfds.disable_progress_bar()


### 6.2 Load Dataset using tensorflow_datasets (tfds)
We can directly download datasets from tensorflow using the package `tensorflow_datasets`, which we installed above.  We can split our dataset into training, testing, and evaluation, as we did so before.

In [ ]:
# Split weights for training/testing/evaluation
SPLIT_WEIGHTS = (8, 1, 1)  # Numbers denote (train, validation, test)

# Split dataset
splits = tfds.Split.TRAIN.subsplit(weighted=SPLIT_WEIGHTS)  # Split according to our weights above

# Load cats_vs_dogs dataset
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs', split=list(splits),
    with_info=True, as_supervised=True)

### 6.3 View Datasets


In [ ]:
# print training data
print("Training data: \n %s \n \n" % (raw_train))
print("Type of training data: \n %s \n \n" % (type(raw_train)))

# print validation data
print("Validation data: \n %s \n \n" % (raw_validation))
print("Type of validation data: \n %s \n \n" % (type(raw_validation)))

# print testing data
print("Test data: \n %s \n \n" % (raw_test))
print("Type of validation data: \n %s \n \n" % (type(raw_validation)))


### 6.4 View Examples from Data

In [ ]:
get_label_name = metadata.features['label'].int2str

# Iteratively show images and labels
for image, label in raw_train.take(2):
  plt.figure()
  plt.imshow(image)
  plt.title(get_label_name(label))
  plt.show()
  plt.clf()

### 6.5 Preprocess Data to Equal Size, and Scale Pixel Values
This is a critical step for developing bug-free and effective datasets for deep learning pipelines.  Here, we will use the `tf.image.resize` function resize the image to the size of `(IMG_SIZE, IMG_SIZE)`.

In [ ]:
IMG_SIZE = 160 # All images will be resized to 160x160

def format_example(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

### 6.6 Create Pre-trained ConvNet Model
Here is where transfer learning will help us for solving our machine learning problem!

In [ ]:
# Create image size to determine input shape of network
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')  # Notice here that we're using weights from ImageNet!

### 6.7 Create Datasets and Specify Hyperparameters

In [ ]:
# Make datasets
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

# Specify hyperparameters
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

# Make training batches
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

### 6.8 Run Feature Extractor

In [ ]:
# Extract image batch from dataset
for image_batch, label_batch in train_batches.take(1):
   print("Inspecting image batch!")

# Inspect an image batch
print("Image batch shape: %s" % (image_batch.shape))

# Call the model on the image batch
feature_batch = base_model(image_batch)
print("Feature batch shape: %s" % (feature_batch.shape))

### 6.9 Get Model Summary and Keep Weights Frozen

In [ ]:
# Let's take a look at the base model architecture
print(base_model.summary())

# Make sure weights are frozen
base_model.trainable = False

### 6.10 Add Final Layers to Transform Features into Predictions, and Stack Into Aggregate Model

In [ ]:
# First do a global averaging
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
# Now make a final prediction layer
prediction_layer = keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
# Finally, build the overall model
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

### 6.11 Compile Model Using Keras and Summarize It

In [ ]:
# Specify learning rate
base_learning_rate = 0.0001

# Compile model using Keras
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Summarize aggregated model
model.summary()

### 6.12 Now We Are Ready to Train!

In [ ]:
num_train, num_val, num_test = (
  metadata.splits['train'].num_examples*weight/10
  for weight in SPLIT_WEIGHTS
)

In [ ]:
initial_epochs = 10
steps_per_epoch = round(num_train)//BATCH_SIZE
validation_steps=20

loss0,accuracy0 = model.evaluate(validation_batches, steps = validation_steps)


In [ ]:
# Now fit the model by training it
history = model.fit(train_batches,
                    epochs=initial_epochs,
                  
                    validation_data=validation_batches)

### 6.13 Plot Evaluation Curves To Visualize Our Performance

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## 7. Processing Images Using TensorFlow
We will provide an example for processing images as input data to TensorFlow.  However, image data is the not the only kind of data TensorFlow can take as input!  TensorFlow is quite useful for taking a variety of other kinds of data, such as time series, audio data, video data, and panel/cross-sectional data.

## 7.5: DataLoaders

## 8. Configuring Devices (CPU and GPU) for TensorFlow
TensorFlow has many capabilities for doing computations on both the CPU and GPU.  GPU computations are another feature of TensorFlow that contribute to its effectiveness as a deployable machine learning packages.

## 9. Conclusion
TensorFlow is a powerful, efficient, and highly-scalable machine learning package for deep learning.  This tutorial covers only a very small fraction of the features and applications of this powerful framework, and we highly encourage you to explore all the capabilities this package offers.  

You can find TensorFlow's online tutorials [here](https://www.tensorflow.org/tutorials).  Note that in addition to you being able to download the Jupyter notebooks that TensorFlow provides and run them on your own machine, you can also run TensorFlow's Jupyter notebooks through [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb).  

We hope this tutorial has provided you with a high-level idea of how TensorFlow works, why it works, and what it can be used for.